# Let's Get Started!

### In case you're having trouble with keras / tensorflow issues, enable (top dropdown menu) following cell and run

#### Import some standard stuff

In [1]:
import config
import math
import numpy as np
import os
import pickle
import sys

#### Import some custom stuff

In [2]:
from keras_model_helpers import build_keras_model, use_lstm
from nb_model_helpers import build_nb_model, use_nb
from util import split_data
from vocabulary_processor_helpers import build_vocabulary_processor, _text_normalizer, _tokenizer

Using TensorFlow backend.


### Process Data

In [3]:
data_file_name = 'data.csv' # change as desired

In [4]:
label_mapping, total_training_samples, total_validation_samples, total_test_samples = split_data(data_file_name)
train_steps = int(math.ceil(total_training_samples / config.batch_size))
validation_steps = int(math.ceil(total_validation_samples / config.batch_size))

with open('label_mapping_dict.p', 'wb') as f:
    pickle.dump(label_mapping, f)
reverse_label_mapping = dict((x[1], x[0]) for x in label_mapping.items())
with open('reverse_label_mapping_dict.p', 'wb') as f:
    pickle.dump(reverse_label_mapping, f)
    

data_root = 'data/'
validation_data_path = os.path.join(data_root, 'validation_data.tsv')
test_data_path = os.path.join(data_root, 'test_data.tsv')
training_data_path = os.path.join(data_root, 'training_data.tsv')

Processing Data. Please Wait!


# Naive Bayes Model

In [5]:
print("Building a Naive Bayes model")
nb_model = build_nb_model(training_data_path, test_data_path, validation_data_path)
nb_probs, true_classes, nb_preds, ids = use_nb(test_data_path)

Building a Naive Bayes model
Processing Training Data
Processing Validation Data
Processing Test Data
                  bank_service  credit_card  credit_reporting  \
bank_service              1567          218                44   
credit_card                131         2369               190   
credit_reporting            29          251              7094   
debt_collection             38          155               478   
loan                        42           82               169   
money_transfers            139           44                 0   
mortgage                    20           24                79   

                  debt_collection  loan  money_transfers  mortgage  
bank_service                   19    32               20       107  
credit_card                   146    73                2        44  
credit_reporting              457   170                1       121  
debt_collection              5097   249                3       127  
loan                          26

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# LSTM Model

In [6]:
print("Building an LSTM model")
vocab_processor = build_vocabulary_processor(training_data_path, config.max_len, config.min_word_count_freq)
with open('vocab_processor.p', 'wb') as f:
    pickle.dump(vocab_processor, f)
    
model = build_keras_model(config.batch_size, config.dropout_rate, config.embedding_size, config.max_len, 
                          config.num_epochs, train_steps, validation_steps, vocab_processor, label_mapping,
                          training_data_path, validation_data_path, model_name='saved_keras_model')

lstm_probs, true_classes, lstm_preds, ids = use_lstm(model, test_data_path, vocab_processor, label_mapping)

Building an LSTM model
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Model built with vocabulary of size 25810
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/2
  37/3355 [..............................] - ETA: 1:45:48 - loss: 1.7791 - acc: 0.2918

KeyboardInterrupt: 

# Can score new datasets and have redicted sample labels will written to tsv files
### Assumes new dataset is in same format as original dataset (data.csv)

#### Change 'dataset_that_you_want_to_score' to the name of your dataset file (should contain same columns/headers/labels as data.csv

In [ ]:
!python use_trained_nb_model.py dataset_that_you_want_score.csv

In [ ]:
!python use_trained_lstm_model.py dataset_that_you_want_score.csv